In [4]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')


from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM

In [5]:
#reference:https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

### Predict future sales btt non time-series models.

In [6]:
ls data/

processed_train.csv    store_item.csv         train.csv
sample_submission.csv  test.csv


In [7]:
df = pd.read_csv('data/processed_train.csv')
df.shape

(913000, 10)

In [8]:
#set datatime to index
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.head()

,store,item,sales,year,month,day,day_of_week,year_and_month,weekend
date,,,,,,,,,
2013-01-01,1,1,13,2013,1,1,1,2013-01,0
2013-01-02,1,1,11,2013,1,2,2,2013-01,0
2013-01-03,1,1,14,2013,1,3,3,2013-01,0
2013-01-04,1,1,13,2013,1,4,4,2013-01,0
2013-01-05,1,1,10,2013,1,5,5,2013-01,1


In [9]:
df['quarter'] = df.index.quarter
df['week_block_num'] = [int(x) for x in np.floor((df.index - pd.to_datetime('2012-12-31')).days/7) + 1]
df['quarter_block_num'] = (df.index.year - 2013) * 4 + df['quarter']

In [10]:
df.head()

,store,item,sales,year,month,day,day_of_week,year_and_month,weekend,quarter,week_block_num,quarter_block_num
date,,,,,,,,,,,,
2013-01-01,1,1,13,2013,1,1,1,2013-01,0,1,1,1
2013-01-02,1,1,11,2013,1,2,2,2013-01,0,1,1,1
2013-01-03,1,1,14,2013,1,3,3,2013-01,0,1,1,1
2013-01-04,1,1,13,2013,1,4,4,2013-01,0,1,1,1
2013-01-05,1,1,10,2013,1,5,5,2013-01,1,1,1,1


In [11]:
df.iloc[500,:]

store                      1
item                       1
sales                     23
year                    2014
month                      5
day                       16
day_of_week                4
year_and_month       2014-05
weekend                    0
quarter                    2
week_block_num            72
quarter_block_num          6
Name: 2014-05-16 00:00:00, dtype: object

In [12]:
df.to_csv('data/lstm_data.csv',index =False)

In [9]:
stores = pd.get_dummies(df['store'], prefix='s',drop_first = True)

In [10]:
stores.head()

,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10
date,,,,,,,,,
2013-01-01,0,0,0,0,0,0,0,0,0
2013-01-02,0,0,0,0,0,0,0,0,0
2013-01-03,0,0,0,0,0,0,0,0,0
2013-01-04,0,0,0,0,0,0,0,0,0
2013-01-05,0,0,0,0,0,0,0,0,0


In [11]:
items = pd.get_dummies(df['item'], prefix='i',drop_first = True)

In [12]:
items.shape,items.head()

((913000, 49),
             i_2  i_3  i_4  i_5  i_6  i_7  i_8  i_9  i_10  i_11  ...  i_41  \
 date                                                            ...         
 2013-01-01    0    0    0    0    0    0    0    0     0     0  ...     0   
 2013-01-02    0    0    0    0    0    0    0    0     0     0  ...     0   
 2013-01-03    0    0    0    0    0    0    0    0     0     0  ...     0   
 2013-01-04    0    0    0    0    0    0    0    0     0     0  ...     0   
 2013-01-05    0    0    0    0    0    0    0    0     0     0  ...     0   
 
             i_42  i_43  i_44  i_45  i_46  i_47  i_48  i_49  i_50  
 date                                                              
 2013-01-01     0     0     0     0     0     0     0     0     0  
 2013-01-02     0     0     0     0     0     0     0     0     0  
 2013-01-03     0     0     0     0     0     0     0     0     0  
 2013-01-04     0     0     0     0     0     0     0     0     0  
 2013-01-05     0     0     0

In [13]:
processed_df = df.copy()

In [ ]:
processed_df =processed_df.join(stores)

In [69]:
def process_product_group(df,groups,prefixs):
    """
    feature engineering:
        get dummies on categorical
    """
    
    for i, group in enumerate(groups):
    #Get the Dummy variables
        dummies = pd.get_dummies(df[group], prefix=prefixs[i],drop_first = True)
        processed_df = df.join(dummies)
    return processed_df

In [70]:
cat_groups =['store','item']
prefixs    =['s','i']

In [ ]:
processed_df = process_product_group(df,cat_groups,prefixs)
processed_df.head()

In [ ]:
#drop unique store & item Id
#drop year & year_and_month
drop_cols =['store','item','year','year_and_month',]

In [ ]:
processed_df.shape

In [18]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(df)

In [20]:
training_set_scaled[:5]

array([[0.        , 0.        , 0.05627706, 0.        , 0.        ,
        0.        , 0.16666667, 0.        ],
       [0.        , 0.        , 0.04761905, 0.        , 0.        ,
        0.03333333, 0.33333333, 0.        ],
       [0.        , 0.        , 0.06060606, 0.        , 0.        ,
        0.06666667, 0.5       , 0.        ],
       [0.        , 0.        , 0.05627706, 0.        , 0.        ,
        0.1       , 0.66666667, 0.        ],
       [0.        , 0.        , 0.04329004, 0.        , 0.        ,
        0.13333333, 0.83333333, 1.        ]])